# Analyzing the Chinook Dataset

In [1]:
import pandas as pd
import sqlite3

def run_query(query):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(query, conn)
    
def run_command(command):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(command)
        
def show_tables():
    query = '''
        SELECT
            name,
            type
        FROM sqlite_master
        WHERE type IN ("table","view");
        '''
    return run_query(query)

show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


## Objective 1:
The Chinook record store has just signed a deal with a new record label, and We've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - here are the artist names, and the genre of music they produce:

&nbsp;

| Artist Name          	| Genre   	|
|----------------------	|---------	|
| Regal                	| Hip-Hop 	|
| Red Tone             	| Punk    	|
| Meteor and the Girls 	| Pop     	|
| Slim Jim Bites       	| Blues   	|

&nbsp;

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so they're interested in finding out which genres sell the best in the USA.

In [2]:
query = '''
    WITH
        invoice_usa_ids AS (
            SELECT invoice_id
            FROM invoice
            WHERE billing_country = 'USA'
        ),
        track_usa_ids AS (
            SELECT track_id
            FROM invoice_line
            INNER JOIN invoice_usa_ids 
            ON invoice_usa_ids.invoice_id = invoice_line.invoice_id
        ),
        genre_usa_ids AS (
            SELECT genre_id
            FROM track
            INNER JOIN track_usa_ids
            ON track_usa_ids.track_id = track.track_id
        ),
        genre_info AS (
            SELECT 
                name,
                COUNT(genre.genre_id) tracks_sold
            FROM genre
            INNER JOIN genre_usa_ids
            ON genre_usa_ids.genre_id = genre.genre_id
            GROUP BY genre.genre_id
        )
    SELECT 
        name,
        tracks_sold,
        ROUND(CAST(tracks_sold AS FLOAT) / (
            SELECT SUM(tracks_sold)
            FROM genre_info
        ) * 100, 2) tracks_sold_percentage
    FROM genre_info
    GROUP BY tracks_sold
    ORDER BY tracks_sold DESC;
    '''

run_query(query)

,name,tracks_sold,tracks_sold_percentage
0,Rock,561,53.38
1,Alternative & Punk,130,12.37
2,Metal,124,11.80
3,R&B/Soul,53,5.04
4,Blues,36,3.43
5,Alternative,35,3.33
6,Pop,22,2.09
7,Hip Hop/Rap,20,1.90
8,Jazz,14,1.33
9,Easy Listening,13,1.24


As we can see here, rock is the most popular genre in the U.S. tracking a little over 50% of all sales. Unfortunately for us, none of the albums within our selection are in the Rock category. Lets display the selections again with the added columns of tracks sold and percent:

&nbsp;

| Artist Name          	| Genre   	| Tracks Sold | Percent |
|----------------------	|---------	| ----------  | ------  |
| Regal                	| Hip-Hop 	| 20          | 1.90%   |
| Red Tone             	| Punk    	| 130         | 12.37%  |
| Meteor and the Girls 	| Pop     	| 22          | 2.09%   |
| Slim Jim Bites       	| Blues   	| 36          | 3.43%   |

&nbsp;

Because we only have to choose 3 out of 4 albums for the store. We'll just exclude the genre with the lowest percentage of tracks sold which is Hip-Hop.

## Objective 2:
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [3]:
query = '''
    SELECT 
        (employee.first_name || ' ' || employee.last_name) name,
        title,
        hire_date,
        SUM(total) total
    FROM employee
    INNER JOIN customer
    ON customer.support_rep_id = employee.employee_id
    INNER JOIN invoice
    ON invoice.customer_id = customer.customer_id
    WHERE title = 'Sales Support Agent'
    GROUP BY name;
    '''
run_query(query)

,name,title,hire_date,total
0,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1731.51
1,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1584.00
2,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1393.92


There seems to be a $200 sales gap between Steve, Margaret, and Jane respectively. Jane has the highest total sales, but she was also hired at the earliest date. Steve has a much better overall performance considering he was hired on almost 5 months after Margaret and 6 months after Jane. 

## Objective 3:
Analyze the sales data for customers from each different country.

In [4]:
query = '''
    WITH
        customer_invoice AS (
            SELECT *
            FROM customer
            INNER JOIN invoice
            ON customer.customer_id = invoice.customer_id
            INNER JOIN invoice_line
            ON invoice.invoice_id = invoice_line.invoice_id
        ),
        ------------------------------------------------------------------------------------
        -- | country | total_sales | avg_customer_value | avg_order_value | total_customer |
        ------------------------------------------------------------------------------------
        country_info AS (
            SELECT 
                country,
                SUM(unit_price) total_sales,
                SUM(unit_price) / COUNT(DISTINCT customer_id) avg_customer_value,
                SUM(unit_price) / COUNT(DISTINCT invoice_id) avg_order_value,
                COUNT(DISTINCT customer_id) total_customer
            FROM customer_invoice
            GROUP BY country
        )
        
    SELECT 
        country_and_other,
        total_customer,
        total_sales,
        avg_customer_value, 
        avg_order_value
    -------------------------------------------------------------------    
    -- put countries with 1 customer into the country column as 'other'
    -------------------------------------------------------------------
    FROM (
        SELECT *,
            CASE
                WHEN total_customer = 1 THEN 'Other' 
                ELSE country
            END AS country_and_other
        FROM country_info
        GROUP BY country_and_other
        ORDER BY total_customer DESC
    );
'''

run_query(query)

,country_and_other,total_customer,total_sales,avg_customer_value,avg_order_value
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,United Kingdom,3,245.52,81.840000,8.768571
6,Czech Republic,2,273.24,136.620000,9.108000
7,India,2,183.15,91.575000,8.721429
8,Portugal,2,185.13,92.565000,6.383793
9,Other,1,75.24,75.240000,7.524000


Czech Republic, India, and Portugal have larger opportunites for growth. The average customer and order values are higher in these regions than in the USA even though the total amount of customers is much smaller.

## Objective 4:
The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [5]:
query = '''
    With
        ----------------------------------------------------
        -- INNER JOIN ON invoice, invoice_line, track, album
        ----------------------------------------------------
        all_joins AS (
            SELECT *
            FROM invoice
            INNER JOIN invoice_line
            ON invoice_line.invoice_id = invoice.invoice_id
            INNER JOIN track
            ON track.track_id = invoice_line.track_id
            INNER JOIN album
            ON album.album_id = track.album_id
        ),
        -------------------
        --| title | price |
        -------------------
        album_price AS (
            SELECT 
                ---------------------------------------------------------
                -- strange bug with the price column, doesn't compare
                -- properly with other numeric values so change to string
                ---------------------------------------------------------
                CAST(SUM(unit_price) AS TEXT) price,
                title
            FROM album
            INNER JOIN track
            ON track.album_id = album.album_id
            GROUP BY title
        ),
        -------------------------------------------------------
        --| invoice_id | title | total | price | album_purchase
        -------------------------------------------------------
        album_purchase_info AS (
            SELECT 
                invoice_id,
                album_price.title,
                total,
                price,
                CASE 
                    WHEN CAST(total AS TEXT) != price THEN 'No'
                    ELSE 'Yes'
                END album_purchase
            FROM all_joins
            INNER JOIN album_price
            ON album_price.title = all_joins.title
            GROUP BY invoice_id
        )
        
    SELECT 
        album_purchase,
        COUNT(*) number_of_invoices,
        CAST(COUNT(*) AS FLOAT) / (
            SELECT COUNT(*)
            FROM album_purchase_info
        ) * 100 AS percentage_of_invoices
    FROM album_purchase_info
    GROUP BY album_purchase;
'''

run_query(query)

,album_purchase,number_of_invoices,percentage_of_invoices
0,No,489,79.641694
1,Yes,125,20.358306


Album purchases account for 20% of all store purchases. The store should continue to buy full albums. A one fifth decrease in revenue would not be worth the possible adjustments.